<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/catboost_20240825.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import copy
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

# warningsを非表示にする
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:

!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.9 MB/s eta 0:00:00


In [4]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [5]:

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

In [6]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_trainV10.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_testV10.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [7]:
df_train.head()

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,MaritalStatus,OwnCar,NumberOfChildren,HasChildren,IncomePerChild,AnnualIncomePerTrip,PitchPlusFollowups,AdultTravelers,ProductPitched_Designation
0,0,50.0,1,2,15.0,2,1,1.0,4.0,4,3.0,5.0,1,4,4.000000,253905.0,1.0,0,0,0.0,0,2.539050e+10,6.093708e+05,23.0,1.0,4_4
1,1,56.0,0,1,14.0,0,1,1.0,4.0,2,3.0,2.0,1,4,2.081575,404475.0,0.0,2,1,0.0,0,4.044750e+10,2.426838e+06,22.0,1.0,2_2
2,2,36.0,1,1,10.0,2,0,1.0,3.0,4,3.0,4.0,0,4,4.000000,278145.0,1.0,1,0,0.0,0,2.781450e+10,8.344329e+05,16.0,1.0,4_4
3,3,37.0,1,2,18.0,1,0,1.0,3.0,2,4.0,1.0,0,5,2.310176,326805.0,0.0,2,1,0.0,0,3.268050e+10,3.921621e+06,24.0,1.0,2_2
4,4,48.0,0,3,17.0,1,0,1.0,3.0,4,4.0,4.0,0,4,4.000000,258435.0,1.0,3,1,0.0,0,2.584350e+10,7.753031e+05,23.0,1.0,4_4


In [8]:
# Config cell
target = "ProdTaken"

In [35]:
# デフォルトの特徴量リスト
default_categorical_features = [
    'TypeofContact', 'CityTier', 'Occupation', 'Gender', 'ProductPitched',
    'PreferredPropertyStar', 'Passport', 'PitchSatisfactionScore',
    'Designation', 'MaritalStatus'
]

default_numerical_features = [
    'id', 'Age', 'DurationOfPitch', 'MonthlyIncome', 'NumberOfPersonVisiting',
    'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren', 'OwnCar'
]

# 追加の数値特徴量
added_numerical_features = [
    'HasChildren', 'IncomePerChild', 'AnnualIncomePerTrip',
    'PitchPlusFollowups', 'AdultTravelers'
]

# 数値特徴量の結合
numerical_features = default_numerical_features + added_numerical_features

# データフレームからすべての特徴量を取得
features = df_train.columns.tolist()

# カテゴリカル特徴量の決定
categorical_features = copy.deepcopy(features)
for feature in numerical_features:
    if feature in categorical_features:
        categorical_features.remove(feature)

print(f'数値特徴量: {numerical_features}')
print(f'カテゴリカル特徴量: {categorical_features}')

# 学習から除外する特徴量の削除
remove_list = ['id', target,'HasChildren']
for item in remove_list:
    if item in features:
        features.remove(item)
    if item in numerical_features:
        numerical_features.remove(item)
    if item in categorical_features:
        categorical_features.remove(item)

print(f'学習に使用する特徴量: {features}')

# カテゴリカル特徴量を整数型に変換
for col in categorical_features:
    df_train[col] = df_train[col].astype(int)
    df_test[col] = df_test[col].astype(int)

数値特徴量: ['id', 'Age', 'DurationOfPitch', 'MonthlyIncome', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren', 'OwnCar', 'HasChildren', 'IncomePerChild', 'AnnualIncomePerTrip', 'PitchPlusFollowups', 'AdultTravelers']
カテゴリカル特徴量: ['TypeofContact', 'CityTier', 'Occupation', 'Gender', 'ProductPitched', 'PreferredPropertyStar', 'Passport', 'PitchSatisfactionScore', 'Designation', 'ProdTaken', 'MaritalStatus', 'ProductPitched_Designation']
学習に使用する特徴量: ['Age', 'TypeofContact', 'CityTier', 'DurationOfPitch', 'Occupation', 'Gender', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'ProductPitched', 'PreferredPropertyStar', 'NumberOfTrips', 'Passport', 'PitchSatisfactionScore', 'Designation', 'MonthlyIncome', 'MaritalStatus', 'OwnCar', 'NumberOfChildren', 'IncomePerChild', 'AnnualIncomePerTrip', 'PitchPlusFollowups', 'AdultTravelers', 'ProductPitched_Designation']


In [36]:
cols_category=categorical_features

In [11]:
train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [12]:

params_cat = {
    "iterations": 1000,  # n_estimatorsに相当
    'eval_metric' : 'AUC',
    'depth':1,
    'l2_leaf_reg' : 8,
    "learning_rate": 0.1,
    "colsample_bylevel": 0.8,  # colsample_bytreeに相当
    'one_hot_max_size' : 25,
    "random_seed": 0,
    "verbose": 100,  # 学習の進捗を100回毎に表示
    "use_best_model": True,  # 早期停止のための設定
}

In [13]:

# f1スコアが最も高くなる点を見つける
from sklearn import metrics
def decide_cutoff(val_y, preds_y_proba):
    mean_f1_list = []
    fpr, tpr, thresholds = metrics.roc_curve(val_y, preds_y_proba)
    for threshold in thresholds:
        preds_y = [1 if prob > threshold else 0 for prob in preds_y_proba]
        #preds_y=preds_y_proba
        mean_f1_list.append(f1_score(val_y, preds_y, average='macro'))
    return np.max(mean_f1_list), thresholds[np.argmax(mean_f1_list)]

In [14]:
!pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 4.2 MB/s eta 0:00:00


In [15]:
import shap

In [38]:
list_metrics_auc = []
list_metrics_f1 = []
list_cutoff = []
list_models = []
fraction_of_positives_list = []
mean_predicted_value_list = []
shap_values_list = []

list_feature_importances = []  # 特徴量重要度を格納するリスト

cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)
for fold, (trn_idx, val_idx) in enumerate(cv.split(train_x, train_y), start=1):
    trn_x = train_x[features].iloc[trn_idx, :]
    trn_y = train_y[trn_idx]
    val_x = train_x[features].iloc[val_idx, :]
    val_y = train_y[val_idx]
    train_pool = Pool(data=trn_x, label=trn_y, cat_features=cols_category)
    val_pool = Pool(data=val_x, label=val_y, cat_features=cols_category)

    model_cat = CatBoostClassifier(**params_cat, early_stopping_rounds=100)
    model_cat.fit(train_pool, eval_set=val_pool)

    list_models.append(model_cat)
    preds_y_proba = model_cat.predict_proba(val_pool)[:, 1]
    auc = roc_auc_score(val_y, preds_y_proba)
    f1, threshold = decide_cutoff(val_y, preds_y_proba)
    list_metrics_auc.append(auc)
    list_metrics_f1.append(f1)
    list_cutoff.append(threshold)
    print(f"Fold: {fold}, AUC: {auc}, f1 score: {f1} Threshold: {threshold}")

    # 特徴量の重要度を追加
    feature_importances = model_cat.get_feature_importance()
    list_feature_importances.append(feature_importances)

# AUCの出力
print("Mean AUC:", np.mean(list_metrics_auc))
print("Mean F1 Score:", np.mean(list_metrics_f1))
print("Median Cutoff:", np.median(list_cutoff))

# 特徴量重要度の集計
mean_feature_importances = np.mean(list_feature_importances, axis=0)

# 結果の表示
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Mean Importance': mean_feature_importances
}).sort_values(by='Mean Importance', ascending=False)

print("Feature Importances:")
print(feature_importance_df)

0:	test: 0.6763525	best: 0.6763525 (0)	total: 1.46ms	remaining: 1.45s
100:	test: 0.8533303	best: 0.8551237 (83)	total: 113ms	remaining: 1.01s
200:	test: 0.8577234	best: 0.8582171 (192)	total: 240ms	remaining: 953ms
300:	test: 0.8566210	best: 0.8586778 (227)	total: 360ms	remaining: 835ms
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8586777675
bestIteration = 227

Shrink model to first 228 iterations.
Fold: 1, AUC: 0.8586777675398184, f1 score: 0.7223370429252782 Threshold: 0.23782246886877859
0:	test: 0.6400059	best: 0.6400059 (0)	total: 1.22ms	remaining: 1.22s
100:	test: 0.8308214	best: 0.8312985 (97)	total: 135ms	remaining: 1.2s
200:	test: 0.8402001	best: 0.8402001 (200)	total: 253ms	remaining: 1.01s
300:	test: 0.8438528	best: 0.8439516 (297)	total: 374ms	remaining: 869ms
400:	test: 0.8453337	best: 0.8454324 (390)	total: 492ms	remaining: 735ms
500:	test: 0.8467487	best: 0.8469133 (485)	total: 613ms	remaining: 611ms
600:	test: 0.8472094	best: 0.8480979 (576)	tot



*   0.8493781249058763 42 k=8
*   0.8483359191672759 1506 K=8
*   0.8491430482256626 999 K-7  
*   0.8498345013282655 618 K=7
*   0.8524340906345512 42 K=7



In [39]:
import pickle

In [40]:
# モデルを保存する
# 全てのモデルを保存する
for fold, model_cat in enumerate(list_models, start=1):
    with open(f'/content/drive/MyDrive/Signate/2024summer/models/catboost_fold_v20{fold}.pkl', 'wb') as f:
        pickle.dump(model_cat, f)

In [41]:


threshold = np.median(list_cutoff)
preds_y_proba = np.zeros(len(df_test))
for model in list_models:
    feature_names = model.feature_names_
    preds_y_proba += model.predict_proba(df_test[feature_names])[:, 1] / len(list_models)
    print(f"Model: {model}, Proba: {preds_y_proba}")  # 予測確率を表示
preds_y = preds_y_proba



Model: <catboost.core.CatBoostClassifier object at 0x7ab33cd33550>, Proba: [0.01387019 0.02937257 0.03424133 ... 0.08241181 0.0367316  0.00129484]
Model: <catboost.core.CatBoostClassifier object at 0x7ab33ccc9c00>, Proba: [0.03007892 0.06279645 0.07259529 ... 0.17119398 0.06871357 0.0020463 ]
Model: <catboost.core.CatBoostClassifier object at 0x7ab33cd33a90>, Proba: [0.041169   0.09369222 0.10725243 ... 0.25425139 0.10885644 0.00526942]
Model: <catboost.core.CatBoostClassifier object at 0x7ab33e2a7fd0>, Proba: [0.05583515 0.13625524 0.14092338 ... 0.35047388 0.14160105 0.00593465]
Model: <catboost.core.CatBoostClassifier object at 0x7ab33e2ff610>, Proba: [0.06743691 0.17298004 0.18070268 ... 0.44573194 0.17473834 0.00693733]
Model: <catboost.core.CatBoostClassifier object at 0x7ab33cb83160>, Proba: [0.07993473 0.20751939 0.21261415 ... 0.52720588 0.20766746 0.00822366]
Model: <catboost.core.CatBoostClassifier object at 0x7ab33cd31300>, Proba: [0.0940314  0.24259297 0.24674543 ... 0.627

In [42]:
count = 90

In [43]:

from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/catboost_{timestamp}_{count:03d}.csv"
ss[1] = preds_y
ss.to_csv(file_name, header=False, index=False)